In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_data_validation as tfdv
import os

from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense


import matplotlib.pyplot as plt

In [127]:
z_df = pd.read_csv("/home/justin/Downloads/emissions/zhvi.csv")
z_df.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,2000-01-31,...,2021-12-31,2022-01-31,2022-02-28,2022-03-31,2022-04-30,2022-05-31,2022-06-30,2022-07-31,2022-08-31,2022-09-30
0,91940,0,77449,zip,TX,TX,NaN,"Houston-The Woodlands-Sugar Land, TX",Harris County,106672.0,...,251930.0,255861.0,260506.0,266174.0,272442.0,278528.0,283555.0,286948.0,288863.0,289504.0
1,91982,1,77494,zip,TX,TX,NaN,"Houston-The Woodlands-Sugar Land, TX",Fort Bend County,223357.0,...,439209.0,445272.0,454755.0,466373.0,480063.0,491447.0,498934.0,501198.0,499334.0,499877.0
2,93144,2,79936,zip,TX,TX,El Paso,"El Paso, TX",El Paso County,90482.0,...,174265.0,176277.0,178252.0,180407.0,183773.0,187384.0,190834.0,193074.0,194779.0,196166.0
3,62080,3,11368,zip,NY,NY,New York,"New York-Newark-Jersey City, NY-NJ-PA",Queens County,187749.0,...,571191.0,569363.0,569160.0,571091.0,572605.0,575407.0,577640.0,579764.0,580707.0,583489.0
4,62093,4,11385,zip,NY,NY,New York,"New York-Newark-Jersey City, NY-NJ-PA",Queens County,275753.0,...,744928.0,748113.0,751397.0,757101.0,759701.0,765523.0,770799.0,775959.0,778630.0,781713.0


In [130]:
emissions_df = pd.read_csv("/home/justin/Downloads/emissions/ghgp_data_by_year.csv", usecols = [3,4,5,13,14,15,16,17,18,19,20,21,22,23], skiprows = [0,1,2])
emissions_df = emissions_df.fillna(0)
emissions_np = emissions_df.to_numpy()
emissions_df.head()
cities = emissions_df['City'].unique()
cities = [str(c).upper() for c in cities]
print(len(cities))

4899


In [107]:
realty_df = pd.read_csv("/home/justin/Downloads/emissions/realtor-data.csv",usecols=[1,7,8,9,11])
realty_df.dropna(subset=['sold_date'])
realty_df = realty_df[(realty_df.sold_date > '2011-01-01') & (realty_df.sold_date < '2022-01-01')]


In [108]:
realty_df.head()


,price,city,state,zip_code,sold_date
684,76900.0,Canovanas,Puerto Rico,729.0,2020-02-28
745,76900.0,Canovanas,Puerto Rico,729.0,2020-02-28
802,76900.0,Canovanas,Puerto Rico,729.0,2020-02-28
829,110000.0,Dorado,Puerto Rico,949.0,2019-06-28
932,76900.0,Canovanas,Puerto Rico,729.0,2020-02-28


In [133]:
realty_train = realty_df.sample(frac=0.5,random_state=200)
realty_test = realty_df.drop(realty_train.index)
realty_train = realty_train.reset_index()
cities1 = realty_train["city"].unique()
cities1 = [c.upper() for c in cities1]
print(len(cities1))

1657


In [134]:
set_dif = set(cities1).symmetric_difference(set(cities))
print(len(set(cities1).difference(set(cities))))

1172


In [79]:
realty_train.drop("index",axis=1)


,price,city,state,zip_code,sold_date
0,750000.0,New York,New York,10021.0,2013-01-22
1,434900.0,Montville Township,New Jersey,7045.0,2017-06-05
2,399888.0,Ringwood,New Jersey,7456.0,2014-01-08
3,1725000.0,Boston,Massachusetts,2108.0,2012-07-17
4,235000.0,Haddon Heights,New Jersey,8035.0,2018-07-03
...,...,...,...,...,...
107077,209999.0,Bronx,New York,10451.0,2017-06-07
107078,899000.0,Tenafly,New Jersey,7670.0,2013-08-09
107079,284900.0,North Providence,Rhode Island,2911.0,2016-08-05
107080,169900.0,Berlin,New Jersey,8009.0,2015-09-29


In [ ]:
#TODO: convert cities to CBSA in order to bring in air quality data from EPA

In [80]:
realty_test = realty_test.reset_index()
realty_test.drop("index", axis=1)


,price,city,state,zip_code,sold_date
0,76900.0,Canovanas,Puerto Rico,729.0,2020-02-28
1,76900.0,Canovanas,Puerto Rico,729.0,2020-02-28
2,76900.0,Canovanas,Puerto Rico,729.0,2020-02-28
3,76900.0,Canovanas,Puerto Rico,729.0,2020-02-28
4,76900.0,Canovanas,Puerto Rico,729.0,2020-02-28
...,...,...,...,...,...
107078,355000.0,New Windsor,New York,12553.0,2019-06-28
107079,225000.0,Highland Mills,New York,10930.0,2017-02-27
107080,569900.0,Cornwall,New York,12518.0,2015-12-21
107081,445000.0,Chappaqua,New York,10514.0,2011-05-09


In [81]:
realty_train_labels_np = realty_train.to_numpy()

In [46]:

def build_model():
    model = keras.Sequential([
        Dense(20, activation=tf.nn.relu, input_shape=[len(realty_train_np[0])]),
        Dense(1)
    ])

    model.compile(optimizer=tf.train.AdamOptimizer(), 
                  loss='mse',
                  metrics=['mae', 'mse'])
    return model



In [47]:
model = build_model()

history = model.fit(train_features, train_labels, epochs=1000, verbose=0, validation_split = 0.1,
                    )

hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch

rmse_final = np.sqrt(float(hist['val_mean_squared_error'].tail(1)))
print()
print('Final Root Mean Square Error on validation set: {}'.format(round(rmse_final, 3)))

InternalError: cudaGetDevice() failed. Status: initialization error